# Install dependencies
This part is in case you run the book in local, because Colab environment already contains most of popular python packages for ML ( like numpy, pandas and sklearn )

In [0]:
!pip install xlsxwriter pandas numpy sklearn matplotlib

# Import dependencies
This par contains all the dependencies that you need to import to run the notebook

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, glob, random, copy, sys, time, re, operator, xlsxwriter
from collections import defaultdict
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold, \
ShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from oauth2client.client import GoogleCredentials
from google.colab import drive

# Mounting your drive 
This part is to mount the drive and be able to save results in it

In [0]:
gdrive_path = '/content/gdrive'
drive.mount(gdrive_path)

# Bee class
This class is the implementation of the Bee Swarm Optimization bee

In [0]:
class Bee :
    def __init__(self,id,problem,locIterations,state):
        self.id=id
        self.data=problem
        self.solution = Solution(self.data,state=state)
        self.fitness= 0.0
        self.rl_return = 0.0
        self.locIterations=locIterations
        self.action = []
    
    def localSearch(self):
        best=self.fitness
        #done=False
        lista=[j for j, n in enumerate(self.solution.get_state()) if n == 1]
        indice =lista[0]
        
        for itr in range(self.locIterations):
            while(True):
                pos = -1
                oldFitness = self.fitness
                for i in range(len(self.solution.get_state())):
                    
                    if ((len(lista)==1) and (indice==i) and (i < self.data.nb_attribs-1)):
                        i+=1
                    self.solution.state[i]= (self.solution.state[i] + 1) % 2
                    quality = self.solution.get_accuracy(self.solution.get_state())
                    
                    if (quality > best):
                        pos = i
                        best = quality
                    self.solution.state[i]= (self.solution.state[i]+1) % 2
                    self.fitness = oldFitness 
                if (pos != -1):
                    self.solution.state[pos]= (self.solution.state[pos]+1)%2
                    self.fitness = best
                else:
                    break
            for i in range(len(self.solution.get_state())):
                oldFitness=self.fitness
                if ((len(lista)==1) and (indice==i) and (i < self.data.nb_attribs-1)):
                    i+=1
                self.solution.state[i]= (self.solution.state[i] + 1) % 2
                quality = self.solution.get_accuracy(self.solution.get_state())
                if (quality<best):
                    self.solution.state[i]= (self.solution.state[i] + 1) % 2
                    self.fitness = oldFitness


    def ql_localSearch(self,maxIterIndex,flip):
      
        """The reason why we do this is to 
        explore at the beginning and 
        eploit at the end to converge to the optimal solution"""
        iterations = int(maxIterIndex/self.locIterations)+1 if int(maxIterIndex/self.locIterations)+1 <= self.locIterations else self.locIterations
        for itr in range(iterations):
       
          state = self.solution.get_state()
          # We get the best solution to be calculated yet
          best_state = Solution.get_best_sol()
          # We xor ( logic xor ) it with the actual state we're in to define the actions that could be done
          if best_state[1] != 0:
            xor_states = Solution.xor(state,best_state[0])
          else:
            xor_states = Solution.xor(state,[0 for i in range(len(state))])

          # We get the indexes of the actions to do and pass them to the step() function to pick the next state  
          actions = Solution.get_indexes(xor_states)

          #next_state, action = self.data.ql.step(self.solution,self.data.nb_attribs)
          # Ths first +1, is not to devide by 0, the 2nd one, is not to get an empty list in case iterations > nb_atts
          #next_state, action = self.data.ql.step(self.solution,int(self.data.nb_attribs/(iterations+1))+1)

          next_state, action = self.data.ql.step(self.solution,actions,flip)
          next_sol = Solution(self.data,state=next_state)
          acc_state = self.solution.get_accuracy(state)
          acc_new_state = self.solution.get_accuracy(next_state)

          if (acc_state < acc_new_state):
              reward = acc_new_state
          elif (acc_state > acc_new_state):
              reward = acc_new_state - acc_state
          else :
              if (Solution.nbrUn(state) > Solution.nbrUn(next_state) ):
                  reward = 1/2 * acc_new_state
              else :
                  reward = -1/2 * acc_new_state

          self.data.ql.learn(self.solution,action,reward,next_sol)
          self.rl_return = self.data.ql.get_q_value(self.solution,action)
          self.fitness = acc_new_state
          self.solution = next_sol
          #print("Next state's acc : ",acc_new_state)
          #print("This is acc choosed : {0}".format(acc_new_state))
            
    def setSolution(self,solution):
        self.solution.set_state(solution)
        self.fitness = self.solution.get_accuracy(solution)
    
    @classmethod
    def Rand(self, num, start=None, end=None): 
        res = [] 
        if (not start) or (not end): 
          """We used 20%/80% (Pareto's law) to initilize the solution"""
          res = np.random.choice([0,1],size=(num,),p=[8./10,2./10]).tolist()
        else: 
          for j in range(num): 
              res.append(random.randint(start, end))
        return res 

# Swarm class
This class is the implementation of the Bee Swarm Optimization swarm

In [0]:
class Swarm :
    def __init__(self,problem,flip,maxChance,nbrBees,maxIterations,locIterations):
        self.data=problem
        self.flip=flip
        self.maxChance=maxChance
        self.nbChance=maxChance
        self.nbrBees=nbrBees
        self.maxIterations=maxIterations
        self.locIterations=locIterations
        self.beeList=[]
        self.refSolution = Bee(-1,self.data,self.locIterations,Bee.Rand(self.data.nb_attribs))
        self.bestSolution = self.refSolution
        self.tabou=[]
        self.feature_count = { i:0 for i in range(self.data.nb_attribs) }
        Solution.solutions.clear()

    def searchArea(self):    
        i=0
        h=0
        
        self.beeList=[]
        while((i<self.nbrBees) and (i < self.flip) ) :
            #print ("First method to generate")
            
            solution=self.refSolution.solution.get_state()
            k=0
            while((self.flip*k+h) < len(solution)):
                solution[self.flip*k +h] = ((solution[self.flip*k+h]+1) % 2)
                k+=1
            newBee=Bee(i,self.data,self.locIterations,solution)
            self.beeList.append(newBee)
            
            i+=1
            h=h+1
        h=0
        
        while((i<self.nbrBees) and (i< 2*self.flip )):
            #print("Second method to generate")

            solution=self.refSolution.solution.get_state()
            k=0
            while((k<int(len(solution)/self.flip)) and (self.flip*k+h < len(solution))):
                solution[int(self.data.nb_attribs/self.flip)*h+k] = ((solution[int(self.data.nb_attribs/self.flip)*h+k]+1)%2)
                k+=1
            newBee=Bee(i,self.data,self.locIterations,solution)
            self.beeList.append(newBee)
            
            i+=1
            h=h+1
        while (i<self.nbrBees):
            #print("Random method to generate")
            solution= self.refSolution.solution.get_state()
            indice = random.randint(0,len(solution)-1)
            solution[indice]=((solution[indice]+1) % 2)
            newBee=Bee(i,self.data,self.locIterations,solution)
            self.beeList.append(newBee)
            i+=1
        for bee in (self.beeList):
            lista=[j for j, n in enumerate(bee.solution.get_state()) if n == 1]
            if (len(lista)== 0):
                bee.setSolution(Bee.Rand(self.data.nb_attribs))
                
    def selectRefSol(self):
      self.beeList.sort(key=lambda Bee: Bee.fitness, reverse=True)
      bestQuality=self.beeList[0].fitness
      if(bestQuality>self.bestSolution.fitness):
          self.bestSolution=self.beeList[0]
          self.nbChance=self.maxChance
          return self.bestSolution
      else:
          if(  (len(self.tabou)!=0) and  bestQuality > (self.tabou[len(self.tabou)-1].fitness)):
              self.nbChance=self.maxChance
              return self.bestBeeQuality()
          else:
              self.nbChance-=1
              if(self.nbChance > 0): 
                  return self.bestBeeQuality()
              else :
                  return self.bestBeeDiversity()
      
    def distanceTabou(self,bee):
        distanceMin=self.data.nb_attribs
        for i in range(len(self.tabou)):
            cpt=0
            for j in range(self.data.nb_attribs):
                if (bee.solution.get_state()[j] != self.tabou[i].solution.get_state()[j]) :
                      cpt +=1
            if (cpt<=1) :
                return 0
            if (cpt < distanceMin) :
                distanceMin=cpt
        return distanceMin
    
    def bestBeeQuality(self):
        
        distance = 0
        i=0
        pos=-1
        while(i<self.nbrBees):
            max_val=self.beeList[i].fitness
            nbUn=Solution.nbrUn(self.beeList[i].solution.get_state())
            while((i<self.nbrBees) and (self.beeList[i].solution.get_accuracy(self.beeList[i].solution.get_state()) == max_val)):
                distanceTemp=self.distanceTabou(self.beeList[i])
                nbUnTemp = Solution.nbrUn(self.beeList[i].solution.get_state())
                if(distanceTemp > distance) or ((distanceTemp == distance) and (nbUnTemp < nbUn)):
                    if((distanceTemp==distance) and (nbUnTemp<nbUn)):
                        print("We pick the solution with less features")
                    nbUn=nbUnTemp
                    distance=distanceTemp
                    pos=i
                i+=1
            if(pos!=-1) :
                return self.beeList[pos]
        bee= Bee(-1,self.data,self.locIterations,Bee.Rand(self.data.nb_attribs))
        return bee
            
    def bestBeeDiversity(self):
        max_val=0
        for i in range(len(self.beeList)):
            if (self.distanceTabou(self.beeList[i])> max_val) :
                max_val = self.distanceTabou(self.beeList[i])
        if (max_val==0):
            bee= Bee(-1,self.data,self.locIterations,Bee.Rand(self.data.nb_attribs))
            return bee
        i=0
        while(i<len(self.beeList) and self.distanceTabou(self.beeList[i])!= max_val) :
            i+=1
        return self.beeList[i]
    
    def bso(self,typeOfAlgo,flip):
        i=1
        while(i<=self.maxIterations):
            t1 = time.time()
            #print("\nrefSolution is : ", Solution.str_sol(self.refSolution.solution.get_state()))
            self.tabou.append(self.refSolution)
            print("BSO iteration N° : ",i)
            
            self.searchArea()

            # The local search part
            
            for j in range(self.nbrBees):
              if (typeOfAlgo == 0):
                self.beeList[j].localSearch()
              elif (typeOfAlgo == 1):
                for episode in range(self.locIterations):
                  self.beeList[j].ql_localSearch(i,flip)
              self.count_features(self.beeList[j].solution.get_state())
              print( "Fitness of bee " + str(j) + " is : " + str(self.beeList[j].fitness) + "\n")
            self.refSolution = self.selectRefSol()
            t2 = time.time()
            print("Time of iteration N°{0} : {1:.2f} s\n".format(i,t2-t1))
            i+=1
            
        print("\n[BSO parameters used]\n")
        print("Type of algo : {0}".format(typeOfAlgo))
        print("Flip : {0}".format(self.flip))
        print("MaxChance : {0}".format(self.maxChance))
        print("Nbr of Bees : {0}".format(self.nbrBees))
        print("Nbr of Max Iterations : {0}".format(self.maxIterations))
        print("Nbr of Loc Iterations : {0}\n".format(self.locIterations))
        print("Must 10% used features : ",self.best_features())
        print("Best solution found : ",self.bestSolution.solution.get_state())
        print("Accuracy of found sol : {0:.2f} ".format(self.bestSolution.fitness*100))
        print("Number of features used : {0}".format(Solution.nbrUn(self.bestSolution.solution.get_state())))
        print("Size of solutions dict : {0}".format(len(Solution.solutions)))
        print("Average time to evaluate a solution : {0:.3f} s".format(Solution.get_avg_time())) 
        print("Global optimum : {0}, {1:.2f}".format(Solution.get_best_sol()[0],Solution.get_best_sol()[1]*100))
        if (typeOfAlgo == 1):
          print("Return (Q-value) : ",self.bestSolution.rl_return)  
          #print("Total sorting time : {0:.2f} s".format(Solution.sorting_time))
        return self.bestSolution.fitness*100, Solution.nbrUn(self.bestSolution.solution.get_state())
      
      
    def count_features(self,solution):
        self.feature_count = {i:self.feature_count[i]+n for i, n in enumerate(solution)}

    def best_features(self):
        sorted_features = sorted(self.feature_count.items(), key=operator.itemgetter(1), reverse=True)
        top_10 = round(0.1*self.data.nb_attribs)+1
        best_features = sorted_features[:top_10]
        return best_features

# Solution class
This class is the implementation of a solution, it contains attributs that a solution could have ( like the subset of featuresor the evaluation, to not train a model with the same subset of features each time ), it could be extensible to other attributs

In [0]:
class Solution:

    solutions = {} 
    best_sol = None
    tot_eval_time = 0
    sorting_time = 0

    def __init__(self,data,state):
        self.data = data
        self.state = state
        self.accuracy = 0
        self.solutions[Solution.str_sol(self.state)] = self.accuracy

    def get_accuracy(self,state):
        if (Solution.str_sol(state) in Solution.solutions):
            if (Solution.solutions[Solution.str_sol(state)] == 0) :
                self.set_accuracy(state)
        else:
          self.set_accuracy(state)

        return Solution.solutions[Solution.str_sol(state)]

    def get_state(self):
        return copy.deepcopy(self.state)

    def set_accuracy(self,state): 
        t1 = time.time()
        Solution.solutions[Solution.str_sol(state)] = self.data.evaluate(state)
        self.accuracy = Solution.solutions[Solution.str_sol(state)]
        t2 = time.time()
        Solution.tot_eval_time += t2-t1
        if (Solution.best_sol == None) or (Solution.best_sol.get_accuracy(Solution.best_sol.get_state()) < self.accuracy):
            Solution.best_sol = self
    
    def set_state(self,state): 
        self.state = copy.deepcopy(state)
            
    @staticmethod
    def get_best_sol():
      # This part has been changed by a variable "best_sol", because sorting was costing some execution time
        """t1 = time.time()
        sorted_sols = sorted(Solution.solutions.items(), key=operator.itemgetter(1), reverse=True)
        t2 = time.time()
        #print("Best sol after sort : {0}".format(sorted_sols[0][1]))
        Solution.sorting_time += t2-t1
        return sorted_sols[0][0] ,sorted_sols[0][1]"""
        if Solution.best_sol == None :
          best_state = Solution.str_sol(list(Solution.solutions.items())[0][0])
          best_accuracy = list(Solution.solutions.items())[0][1]
        else:
          best_state = Solution.best_sol.get_state()
          best_accuracy = Solution.best_sol.get_accuracy(best_state)
        return Solution.str_sol(best_state), best_accuracy

      
    @staticmethod
    def get_indexes(mlist):
        ilist = []
        for i in range(len(mlist)):
          if mlist[i] == 1:
            ilist.append(i)
        return ilist
      
    @staticmethod
    def str_sol(mlist):
        result = ''
        for element in mlist:
            result += str(element)
        return result
    
    @staticmethod
    def sol_to_list(solution):
        sol_list=[i for i, n in enumerate(solution) if n == 1]
        return sol_list

    @staticmethod
    def list_sol(key):
        mlist = [ int(i) for i in key ]
        return mlist
      
    @staticmethod
    def nbrUn(state):
        return len([i for i, n in enumerate(state) if n == 1])
    
    @staticmethod
    def attributs_to_flip(nb_att):
        return list(range(nb_att))
    
    @staticmethod
    def xor(x, y):
        return '{1:0{0}b}'.format(len(x), int(Solution.str_sol(x), 2) ^ int(Solution.str_sol(y), 2))
    
    @staticmethod
    def get_avg_time():
        return Solution.tot_eval_time/len(Solution.solutions)

# Reinforcement learning class
This class containts the implementation of the different RL algorithms

In [0]:
class QLearning:
    def __init__(self,nb_atts,actions,alpha=0.1,gamma=0.9,epsilon=0.1):
        self.actions = actions
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = [ {} for i in range(nb_atts) ] 

    def get_max_q_value(self,solution,actions_vals):
        max_val = 0.0
        arg_max = 0

        for i in actions_vals: # Basic itirative max search from a list of possible actions
            state_i = self.get_next_state(solution,i)
            state_i_acc = solution.get_accuracy(state_i)
            if state_i_acc > max_val: 
                max_val = self.get_q_value(solution,i) + state_i_acc
                arg_max = i

        return max_val, arg_max # We return the max q_value and the action that led to it from that state


    def get_q_value(self,solution,action):
        
        state = solution.get_state()
        if not Solution.str_sol(state) in self.q_table[Solution.nbrUn(state)]: 
            self.q_table[Solution.nbrUn(state)][Solution.str_sol(state)] = {}

        if not str(action) in self.q_table[Solution.nbrUn(state)][Solution.str_sol(state)]:
            # We initilize the q_table with 0
            self.q_table[Solution.nbrUn(state)][Solution.str_sol(state)][str(action)] = 0
            
        return self.q_table[Solution.nbrUn(state)][Solution.str_sol(state)][str(action)]

    
    def set_q_value(self,solution,action,val):
        state = solution.get_state()
        self.q_table[Solution.nbrUn(state)][Solution.str_sol(state)][str(action)] = val

    
    def step(self,solution,actions,flip):
        
        if len(actions) != 0:
          if len(actions) > flip:
            self.actions = [actions[i] for i in sorted(random.sample(range(len(actions)), flip))]
          else:
            self.actions = [actions[i] for i in sorted(random.sample(range(len(actions)), 1))]
        
        if np.random.uniform() > self.epsilon :
            
            #action_values = [self.actions[i] for i in sorted(random.sample(range(len(self.actions)), sample_size))]
            action_values = self.actions
            max_val = self.get_max_q_value(solution,action_values)[0] # getting the max next q_value
            argmax_actions=[self.get_max_q_value(solution,action_values)[1]] # saving the action that maxmizes the reward

            # There may be actions that have the same reward, so we add them to the argmax_avtions
            for ac in action_values : 
              ac_state = self.get_next_state(solution,ac)
              ac_state_q_val = self.get_q_value(solution,ac) + solution.get_accuracy(ac_state)
              
              if ( ac_state_q_val >= max_val ):
                  argmax_actions.append(ac) 
                  # We could make the condition "equal", because theorically there won't be any bigger q_value
            next_action = np.random.choice(argmax_actions) # We choose a random action from eqaul reward actions
            next_state = self.get_next_state(solution,next_action)

        else : # This is the exploration condition
            next_action = np.random.choice(self.actions)
            next_state = self.get_next_state(solution,next_action)

        if self.epsilon > 0 :
            self.epsilon -= 0.0001 
        if self.epsilon < 0 :
            self.epsilon = 0

        return next_state, next_action 

    def get_next_state(self,solution,action):
        next_state = solution.get_state()
        next_state[action] = (next_state[action]+1) % 2
        if (Solution.nbrUn(next_state) != 0):
          return next_state
        else:
          return solution.get_state()
    
    def learn(self,current_sol,current_action,reward,next_sol):
        next_action = self.get_max_q_value(next_sol,self.actions)[1] # Get the action with the max reward
        new_q = reward + self.gamma * self.get_q_value(next_sol,next_action)  #This part will be multiplied by alpha
        self.set_q_value(current_sol,current_action,(1 - self.alpha)*self.get_q_value(current_sol,current_action) + self.alpha*new_q) # This is the basic Q-learning formula
        


# Evaluation class
This class contains the implementation of evaluation methods and could be extended to other problems than feature selection

In [0]:
class FsProblem :
    def __init__(self,typeOfAlgo,data,qlearn,classifier=KNeighborsClassifier(n_neighbors=1)):
        self.data=data
        self.nb_attribs = len(self.data.columns)-1 # The number of features is the size of the dataset - the 1 column of labels
        self.outPuts=self.data.iloc[:,self.nb_attribs] # We initilize the labels from the last column of the dataset
        self.ql = qlearn
        self.classifier = classifier
        self.typeOfAlgo = typeOfAlgo

    def evaluate2(self,solution):
        sol_list = Solution.sol_to_list(solution)
        if (len(sol_list) == 0):
            return 0
         
        df = self.data.iloc[:,sol_list]
        array=df.values
        X = array[:,0:self.nb_attribs]
        Y = self.outPuts
        train_X, test_X, train_y, test_y = train_test_split(X, Y, 
                                                    random_state=0,
                                                    test_size=0.1
                                                    )
        self.classifier.fit(train_X,train_y)
        predict= self.classifier.predict(test_X) 
        return metrics.accuracy_score(predict,test_y)


    def evaluate(self,solution):
        sol_list = Solution.sol_to_list(solution)
        if (len(sol_list) == 0):
            return 0
        
        df = self.data.iloc[:,sol_list] # For this function you need to put the indexes of features you picked  
        array=df.values
        X = array[:, 0:self.nb_attribs]
        Y = self.outPuts
        cv = ShuffleSplit(n_splits=10, test_size=0.1, random_state=0) # Cross validation function
        results = cross_val_score(self.classifier, X, Y, cv=cv,scoring='accuracy')
        #print("\n[Cross validation results]\n{0}".format(results))
        return results.mean()



# Data class
This class is the one resbonsible for processing and getting the data ready to be trained

In [0]:
class FSData():

    def __init__(self,typeOfAlgo,location,nbr_exec, method, test_param, param, val, classifier, alpha=None,gamma=None,epsilon=None):
        
        self.typeOfAlgo = typeOfAlgo
        self.location = location
        self.nb_exec = nbr_exec
        self.dataset_name = re.search('[A-Za-z\-]*.csv',self.location)[0].split('.')[0]
        self.df = pd.read_csv(self.location,header=None)
        self.ql = QLearning(len(self.df.columns),Solution.attributs_to_flip(len(self.df.columns)-1),alpha,gamma,epsilon)
        self.fsd = FsProblem(self.typeOfAlgo,self.df,self.ql)
        
        self.classifier_name = str(type(self.fsd.classifier)).strip('< > \' class ').split('.')[3]
        path = gdrive_path + '/My Drive/Colab Notebooks/results/parameters/'+method+'/'+test_param+'/'+param+'/'+val+'/'+classifier+'/'+ self.dataset_name
        if not os.path.exists(path):
          os.makedirs(path + '/logs/')
          os.makedirs(path + '/sheets/')
        self.instance_name = self.dataset_name + '_' +  str(time.strftime("%m-%d-%Y_%H-%M-%S_", time.localtime()) + self.classifier_name)
        log_filename = str(path + '/logs/'+ self.instance_name)
        if not os.path.exists(path):
          os.makedirs(path)
        log_file = open(log_filename + '.txt','w+')
        sys.stdout = log_file
        
        print("[START] Dataset " + self.dataset_name + " description \n")
        print("Shape : " + str(self.df.shape) + "\n")
        print(self.df.describe())
        print("\n[END] Dataset " + self.dataset_name + " description\n")
        print("[START] Ressources specifications\n")
        !cat /proc/cpuinfo # Think of changing this when using Windows
        print("[END] Ressources specifications\n")

        
        sheet_filename = str(path + '/sheets/'+ self.instance_name )
        self.workbook = xlsxwriter.Workbook(sheet_filename + '.xlsx')
        
        self.worksheet = self.workbook.add_worksheet(self.classifier_name)
        self.worksheet.write(0,0,"Iteration")
        self.worksheet.write(0,1,"Accuracy")
        self.worksheet.write(0,2,"N_Features")
        self.worksheet.write(0,3,"Time")
        self.worksheet.write(0,4,"Top_10%_features")
        self.worksheet.write(0,5,"Size_sol_space")

    
    def run(self,flip,maxChance,nbrBees,maxIterations,locIterations):
        total_time = 0
        
        for itr in range(1,self.nb_exec+1):
          print ("Execution {0}".format(str(itr)))
          self.fsd = FsProblem(self.typeOfAlgo,self.df,self.ql)
          swarm = Swarm(self.fsd,flip,maxChance,nbrBees,maxIterations,locIterations)
          t1 = time.time()
          best = swarm.bso(self.typeOfAlgo,flip)
          t2 = time.time()
          total_time += t2-t1
          print("Time elapsed for execution {0} : {1:.2f} s\n".format(itr,t2-t1))
          self.worksheet.write(itr, 0, itr)
          self.worksheet.write(itr, 1, round(best[0],2))
          self.worksheet.write(itr, 2, best[1])
          self.worksheet.write(itr, 3, round(t2-t1,3))
          self.worksheet.write(itr, 4, "{0}".format(str([j[0] for j in [i for i in swarm.best_features()]])))
          self.worksheet.write(itr, 5, len(Solution.solutions))
          
        print ("Total execution time of {0} executions \nfor dataset \"{1}\" is {2:.2f} s".format(self.nb_exec,self.dataset_name,total_time))
        self.workbook.close()


# Visualization class
This class is responsible of plotting visuals out of input data.

In [0]:
"""class Plot:
  def __init__(self,data_file):"""

# The main program
In the part you can specify the parameters' values.
Those parameters are :


*   **dataset** : the dataset name ( without **.cvs** extension ), from the list of [datasets](https://github.com/Neofly4023/bso-fs/tree/master/datasets), or you can put your own **.csv** dataset (without header row, and index column)
*   **typeOfAlgo** : for now, the value is 0, for the original localsearch algorithm, or 1, for the q-localsearch algorithm
*  **nbr_exec** : the number of executions
*  **flip** : the flip parameter is used to generate solutions from the searchArea
*  **maxChance** : the maxChance parameter is a used for exploitation
*  **nbrBees** : the number of Bees used
*  **maxIterations** : the number of iterations inside the BSO algorithm, it is possible to not reach it
*  **locIterations** : the number of iterations inside the localsearch algorithms (orginal & q-learning)

For more details, please check the previous [work](https://link.springer.com/chapter/10.1007%2F978-3-319-19258-1_33), or PM me via : [e-mail](mailto:ea_remache@esi.dz)

In [0]:
# Main program

# RL 

alpha = 0.1
gamma = 0.99
epsilon = 0.1

# BSO

flip = 5
maxChance = 3
nbrBees = 10
maxIterations = 10
locIterations = 10

# Test type

typeOfAlgo = 1
nbr_exec = 10
dataset = "Glass"
data_loc_path = "https://raw.githubusercontent.com/Neofly4023/bso-fs/master/datasets/"
location = data_loc_path + dataset + ".csv"
method = "qbso_simple"
test_param = "rl"
param = "gamma"
val = str(locals()[param])
classifier = "knn"

instance = FSData(typeOfAlgo,location,nbr_exec,method,test_param,param,val,classifier,alhpa,gamma,epsilon)
instance.run(flip,maxChance,nbrBees,maxIterations,locIterations)